<a href="https://colab.research.google.com/github/MohammadDeep/vision/blob/main/modeles/model_1_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# model_1

In [ ]:
import torch
from torch import nn
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.optim as optim

# 1. بارگذاری مدل پیش‌آموزش‌دیده SqueezeNet
model_1 = models.squeezenet1_1(pretrained=True)

# 2. تغییر لایه آخر مدل به یک نود (برای باینری: وجود یا عدم وجود انسان)
model_1.classifier[1] = nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))

# اضافه کردن Softmax برای تبدیل خروجی به احتمال باینری
# این مرحله در اینجا مورد نیاز نیست، چون BCEWithLogitsLoss به‌طور داخلی سیگموید را اعمال می‌کند.

model_1.classifier = nn.Sequential(
    model_1.classifier[0],
    model_1.classifier[1],
    nn.Conv2d(in_channels=1, out_channels=1, kernel_size=13, stride=1, padding=0),
    #nn.AdaptiveAvgPool2d(1),
    nn.Flatten()
)
for param in model_1.features.parameters():
    param.requires_grad = False  # غیرفعال کردن آموزش برای لایه‌های اولیه


In [ ]:
# بررسی اینکه آیا GPU موجود است یا نه
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# انتقال مدل به دستگاه (GPU یا CPU)
model_1.to(device)


# نمایش خلاصه مدل
from torchsummary import summary
summary(model_1, (3, 224, 224))  # برای ورودی‌های 64x64 پیکسل با 3 کانال رنگ


## transform and dataloaber

In [ ]:
from torchvision import datasets, transforms
# 4. آماده‌سازی داده‌ها
transform = transforms.Compose([
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),   # تغییر اندازه به ورودی مدل
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


BATCH_SIZE = 2 ** 7
# ImageFolder دو کلاس را 0 و 1 نگاشت می‌کند
dataset_val = datasets.ImageFolder(root=dir_folber_val, transform=transform)
dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

print(dataset_val.class_to_idx)

# e.g. {'negative': 0, 'positive': 1}
# ImageFolder دو کلاس را 0 و 1 نگاشت می‌کند
dataset_train = datasets.ImageFolder(root=dir_folber_train, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

print(dataset_train.class_to_idx)

## loss and optimiser

In [ ]:

# 3. تعریف تابع از دست دادن (Loss) و بهینه‌ساز
# استفاده از BCEWithLogitsLoss که برای خروجی باینری سیگموید استفاده می‌شود
loss_function_1 = nn.BCEWithLogitsLoss()  # این تابع از سیگموید به‌طور داخلی استفاده می‌کند
optimizer_1 = optim.Adam(model_1.parameters(), lr=0.001)




## training

In [ ]:
model_1_results = train(model=model_1,
                        train_dataloader=dataloader_train,
                        test_dataloader=dataloader_val,
                        optimizer=optimizer_1,
                        loss_fn=loss_function_1,
                        epochs = 1,
                        number_ep = 100)